# House Price Prediction

# 1. Background:
    Of course, every one wants to have their own dream house, but people need to consider  lots of conditions, like location of house, house quality, whethere any school near the neiborhood etc.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## scipy
from scipy import stats

##xgboost
import xgboost
from xgboost import plot_importance

## matplotlib
import seaborn as sns; sns.set() # data visulization
import matplotlib.pyplot as plt # data visulization
%matplotlib inline

from collections import OrderedDict

## plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## sklearn
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# 2. Prepare the data
## 2.1 Load data from csv 

In [ ]:
## read train.csv file into a pd dataframe named as df_train
df_train = pd.read_csv('../input/train.csv')
print('done')

In [ ]:
## read test.csv file into a pd dataframe named as df_test
df_test = pd.read_csv('../input/test.csv')
print('done')

## 2.2 Understand Dataset

### 2.2.1 Take a look of all features

In [ ]:
## print out all of the colums in df_train
df_train.columns

In [ ]:
## print out all of the colums in df_test
df_test.columns

### 2.2.2 Take a look of dataset size

In [ ]:
## print out shape
print(df_train.drop_duplicates().shape)
print(df_test.drop_duplicates().shape)


### 2.2.3 Describe data

In [ ]:
## describe df_train
df_train.describe()

In [ ]:
## print first 5 rows of df_train
df_train.head(5)

In [ ]:
print(df_train.info)

In [ ]:
## describe df_test
df_test.describe()

In [ ]:
## print first 5 rows of df_test
df_test.head(5)

In [ ]:
print(df_test.info)

# 3. Data processing

## 3.1 Data clean
### 3.1.1 Drop 'ID'

In [ ]:
df_train.drop('Id',axis=1,inplace=True)
df_test.drop('Id',axis=1,inplace=True)

### 3.1.2 Drop columns which percentage of missing value is greater than 15%

* Trainning Data

In [ ]:
## count how many missing value which features have
missing_total = df_train.isnull().sum().sort_values(ascending = False)

## calculate percentage of missing data of each feature
percent = (missing_total / df_train.isnull().count()).sort_values(ascending = False)

## combine total numbers and percent into a dataframe
percent_missing_data = pd.concat([missing_total,percent],axis=1,keys=['Total','Percent'],sort=False)

## assign a continus numbers to dataframe as Index to order to plot text value from each features in barplot
Index = range(0,80)
percent_missing_data['Index'] = Index

## print first 20 rows of data
percent_missing_data.head(20)

In [ ]:
## print barplot showing percentage of each feature

## initilize figure size
f,ax = plt.subplots(figsize = (10,7))

## rotate X ticks in case X ticks overhead
plt.xticks(rotation='90')

## choose first 20 rows assigning to barplot
sns.barplot(x=percent_missing_data.index[:20], y=percent_missing_data.iloc[:20,1]*100)

## assign X,Y label 
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)

## assign plot title
plt.title('Percent missing data of Training Data by feature', fontsize=20)

## assign percentage value to each feature
for index, row in percent_missing_data.iloc[:20].iterrows():
    plt.text(int(row.Index),(row.Percent * 100) + 0.1,(np.around(row.Percent * 100,decimals=2)).astype(str) + "%",ha = "center",color = "black" )

## show plot
plt.show()

In [ ]:
## drop feature which percentage of missing value > 15%
df_train_drop_useless = df_train.drop(percent_missing_data[percent_missing_data['Percent'] > 0.15].index,axis=1)

## check size of data after dropping
print(df_train_drop_useless.shape)

* Testing Data

In [ ]:
## count how many missing value which features have
missing_total_test = df_test.isnull().sum().sort_values(ascending = False)

## calculate percentage of missing data of each feature
percent_test = (missing_total_test / df_test.isnull().count()).sort_values(ascending = False)

## combine total numbers and percent into a dataframe
percent_missing_data_test = pd.concat([missing_total_test,percent_test],axis=1,keys=['Total','Percent'],sort=False)

## assign a continus numbers to dataframe as Index to order to plot text value from each features in barplot
Index = range(0,79)
percent_missing_data_test['Index'] = Index

## print first 20 rows of data
percent_missing_data_test.head(25)

In [ ]:
## print barplot showing percentage of each feature

## initilize figure size
f,ax = plt.subplots(figsize = (13,7))

## rotate X ticks in case X ticks overhead
plt.xticks(rotation='90')

## choose first 20 rows assigning to barplot
sns.barplot(x=percent_missing_data_test.index[:25], y=percent_missing_data_test.iloc[:25,1]*100)

## assign X,Y label 
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)

## assign plot title
plt.title('Percent missing data of Testing Data by feature', fontsize=20)

## assign percentage value to each feature
for index, row in percent_missing_data_test.iloc[:25].iterrows():
    plt.text(int(row.Index),(row.Percent * 100) + 0.1,(np.around(row.Percent * 100,decimals=2)).astype(str) + "%",ha = "center",color = "black" )

## show plot
plt.show()

In [ ]:
## drop feature which percentage of missing value > 15%
df_test_drop_useless = df_test.drop(percent_missing_data_test[percent_missing_data_test['Percent'] > 0.15].index,axis=1)

## check size of data after dropping
print(df_test_drop_useless.shape)

### 3.1.3 Fill missing value

* **Dealing with training Data**

In [ ]:
## figure out why GarageType is missing
df_train_missing_Garage = df_train[df_train['GarageCond'].isnull() == True]
df_train_missing_Garage

In [ ]:
df_train_missing_Garage[['GarageCond','GarageType','GarageQual','GarageFinish','GarageYrBlt']]

According to datafrme showing below, I find missing data comes when a house doesn't have any garage.  
* **GargeCond,GarageType,GarageQual and GarageFinish**: Replace missing data with None

In [ ]:
for col in ('GarageCond','GarageType','GarageQual','GarageFinish'):
    df_train_drop_useless[col] = df_train_drop_useless[col].fillna('None')

*** **GarageYrBlt**: Replace missing data with 0

In [ ]:
df_train_drop_useless['GarageYrBlt'] = df_train_drop_useless['GarageYrBlt'].fillna(0)

In [ ]:
## figure out why Basement value is missing
df_train_missing_Basement = df_train[df_train['BsmtCond'].isnull() == True]

In [ ]:
df_train_missing_Basement[['BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','BsmtExposure']]

According to the result showing below: I find the missing data comes when a house doesn't have a basement.
* **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 and BsmtFinType2**: Replace missing data with None

In [ ]:
for col in ('BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','BsmtExposure'):
    df_train_drop_useless[col] = df_train_drop_useless[col].fillna('None')

In [ ]:
## figure out why Masonry veneer value is missing
df_train_missing_Masonry = df_train[df_train['MasVnrType'].isnull() == True]

In [ ]:
df_train_missing_Masonry[['MasVnrType','MasVnrArea']]

According to the result showing below: I find the missing data comes when a house doesn't have any Masonry veneer.
* **MasVnrType**: Replace missing data with None

In [ ]:
df_train_drop_useless['MasVnrType'] = df_train_drop_useless['MasVnrType'].fillna('None')

* **MasVnrArea**: Replace missing data with 0

In [ ]:
df_train_drop_useless['MasVnrArea'] = df_train_drop_useless['MasVnrArea'].fillna(0)

In [ ]:
## figure out why Electrical value is missing
df_train[df_train['Electrical'].isnull() == True]

In [ ]:
## count value
df_train['Electrical'].value_counts()

* **Electrical**: Replace missing data with 'SBrkr', because this feature has mostly 'SBrkr'

In [ ]:
df_train_drop_useless['Electrical'] = df_train_drop_useless['Electrical'].fillna('SBrkr')

In [ ]:
## double check training dataset
df_train_drop_useless.isnull().sum().max()

* Dealing with testing data

In [ ]:
## figure out why GarageType is missing
df_test_missing_Garage = df_test[df_test['GarageCond'].isnull() == True]

In [ ]:
df_test_missing_Garage[['GarageCond','GarageType','GarageQual','GarageFinish','GarageYrBlt']]

According to datafrme showing below, I find missing data comes when a house doesn't have any garage.

* **GargeCond,GarageType,GarageQual and GarageFinish**: Replace missing data with None

In [ ]:
for col in ('GarageCond','GarageType','GarageQual','GarageFinish'):
    df_test_drop_useless[col] = df_test_drop_useless[col].fillna('None')

* **GarageYrBlt**: Replace missing data with 0

In [ ]:
df_test_drop_useless['GarageYrBlt'] = df_test_drop_useless['GarageYrBlt'].fillna(0)

In [ ]:
## figure out why Basement value is missing
df_test_missing_Basement = df_test[df_test['BsmtCond'].isnull() == True]

In [ ]:
df_test_missing_Basement[['BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','BsmtExposure']]

According to the result showing below: I find the missing data comes when a house doesn't have a basement.

* **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 and BsmtFinType2**: Replace missing data with None

In [ ]:
for col in ('BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','BsmtExposure'):
    df_test_drop_useless[col] = df_test_drop_useless[col].fillna('None')

* figure out why Masonry veneer value is missing

In [ ]:
df_test_missing_Masonry = df_test[df_test['MasVnrType'].isnull() == True]

In [ ]:
df_test_missing_Masonry[['MasVnrType','MasVnrArea']]

According to the result showing below: I find the missing data comes when a house doesn't have any Masonry veneer.

* **MasVnrType**: Replace missing data with None

In [ ]:
df_test_drop_useless['MasVnrType'] = df_test_drop_useless['MasVnrType'].fillna('None')

* **MasVnrArea**: Replace missing data with 0

In [ ]:
df_test_drop_useless['MasVnrArea'] = df_test_drop_useless['MasVnrArea'].fillna(0)

In [ ]:
## figure out why MS Zoning value is missing
df_test[df_test['MSZoning'].isnull() == True]

In [ ]:
## count value
df_test['MSZoning'].value_counts()

* **MS Zoning**: Replace missing data with 'RL', because this feature has mostly 'RL'

In [ ]:
df_test_drop_useless['MSZoning'] = df_test_drop_useless['MSZoning'].fillna('RL')

* Drop all of other missing data because there are only 1 or 2 in whole dataset considers as outlier.

In [ ]:
df_test_drop = df_test_drop_useless.dropna()

In [ ]:
## double check testing dataset
df_test_drop.isnull().sum().max()

### 3.1.4. Transform some numercial features into their real datatype

#MSSubClass=The building class
df_train_drop_useless['MSSubClass'] = df_train_drop_useless['MSSubClass'].apply(str)

#Changing OverallCond,OverallQual into a categorical variable
df_train_drop_useless['OverallCond'] = df_train_drop_useless['OverallCond'].astype(str)
df_train_drop_useless['OverallQual'] = df_train_drop_useless['OverallQual'].astype(str)

#Year and month sold are transformed into categorical features.
df_train_drop_useless['YrSold'] = df_train_drop_useless['YrSold'].astype(str)
df_train_drop_useless['MoSold'] = df_train_drop_useless['MoSold'].astype(str)
df_train_drop_useless['YearBuilt'] = df_train_drop_useless['YearBuilt'].astype(str)
df_train_drop_useless['YearRemodAdd'] = df_train_drop_useless['YearRemodAdd'].astype(str)
df_train_drop_useless['GarageYrBlt'] = df_train_drop_useless['GarageYrBlt'].astype(str)
df_train_drop_useless['GarageCars'] = df_train_drop_useless['GarageCars'].astype(str)

## 3.2 Beyond Analytics:Understanding value of the Data)

### 3.2.1. Understanding label(sale price)

In [ ]:
## overview SalePrice
df_train_drop_useless['SalePrice'].describe()

In [ ]:
## plot distribution plot
sns.distplot(df_train_drop_useless['SalePrice'],kde_kws={"color":"black","label":"KDE"})

## plot probability plot
fig = plt.figure()
res = stats.probplot(df_train_drop_useless['SalePrice'],plot=plt)
plt.show()

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df_train_drop_useless['SalePrice'].skew())
print("Kurtosis: %f" % df_train_drop_useless['SalePrice'].kurt())

* Normalize Sale Price

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
df_train_drop_useless["SalePrice"] = np.log1p(df_train_drop_useless["SalePrice"])

#Check the new distribution 
sns.distplot(df_train_drop_useless['SalePrice']);

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(df_train_drop_useless['SalePrice'], plot=plt)
plt.show()

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df_train_drop_useless['SalePrice'].skew())
print("Kurtosis: %f" % df_train_drop_useless['SalePrice'].kurt())

### 3.2.2.Understanding numerical features

* Ground Living Area

In [ ]:
## plot Ground living area with dist plot 
f,ax = plt.subplots(figsize=(5,5))
sns.distplot(df_train_drop_useless['GrLivArea'],kde_kws={"color":"black","label":"KDE"})

## also plot probability plot
fig = plt.figure()
res =  stats.probplot(df_train_drop_useless['GrLivArea'],plot = plt)
plt.show()

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df_train_drop_useless['GrLivArea'].skew())
print("Kurtosis: %f" % df_train_drop_useless['GrLivArea'].kurt())

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
df_train_drop_useless["GrLivArea"] = np.log1p(df_train_drop_useless["GrLivArea"])

#Check the new distribution 
sns.distplot(df_train_drop_useless['GrLivArea']);

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(df_train_drop_useless['GrLivArea'], plot=plt)
plt.show()

* plot distribution dist plot of GarageArea feature

In [ ]:
## plot GarageArea with dist plot 
f,ax = plt.subplots(figsize=(5,5))
sns.distplot(df_train_drop_useless['GarageArea'],kde_kws={"color":"black","label":"KDE"})

## also plot probability plot
fig = plt.figure()
res =  stats.probplot(df_train_drop_useless['GarageArea'],plot = plt)
plt.show()

print("skewness: {}".format(df_train_drop_useless['GarageArea'].skew()))
print("kurtosis: {}".format(df_train_drop_useless['GarageArea'].kurt()))


### 4. Feature engineering
#### 4.1 Feature selection

#### 4.1.1 Data filtering
* cut off by Pearson correlation coefficient

In [ ]:
## show relationship among all of the feature by using heatmap
#correlation matrix
corrmat = df_train_drop_useless.corr()

f,ax = plt.subplots(figsize=(10,10))
sns.heatmap(corrmat)

In [ ]:
data = corrmat.sort_values(by=['SalePrice'],ascending=False)
x = data['SalePrice']
y = data.index
## bar plot showing more clear
f,ax = plt.subplots(figsize = (8,8))
sns.barplot(x,y)
sns.set_color_codes('pastel')
plt.xlabel('Correlation coefficient')
plt.title('Pearson correlation coefficient')

In [ ]:
## compare corrlation value between sale price and all others feartures, figure out how many fearture moderate relationship with sale price(out put)
select_pos = corrmat['SalePrice'] >= 0.4  
select_nag = corrmat['SalePrice'] <= -0.4
select_pos = select_pos[select_pos == True]
print(select_pos.count()-1)
select_nag = select_nag[select_nag == True]
print(select_nag.count()-1)

In [ ]:
index = corrmat.nlargest(12,'SalePrice')['SalePrice'].index
index

* Pearson Correlation Heatmap

In [ ]:
## show relationship among top 11 features by using heatmap
corr_after = df_train_drop_useless[index].corr()
corr_after


In [ ]:
## plot heatmap
colormap = plt.cm.RdBu_r
f,ax = plt.subplots(figsize=(10,10))
plt.title('Pearson Correlation of each related feature',size = 15)
sns.heatmap(corr_after,linecolor='white',linewidths=0.1,cmap=colormap,annot=True,vmax=1.0,vmin=0)

#### 4.1.2 Data wrapping
* cut off by Random forest

In [ ]:
x = df_train_drop_useless.iloc[:,0:-1]
y = df_train_drop_useless.iloc[:,-1]

## one-hot encoding categorical features
x_dummies = pd.get_dummies(x)

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_dummies,y)
importance = rf.feature_importances_
indices = np.argsort(importance)[::-1]
features = x_dummies.columns
for f in range(0,30):
 print(("%2d) %-*s %f" % (f + 1, 30, features[f], importance[indices[f]])))

* cut off by xgboost regressor

model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

every_column_except_y= [col for col in df_train_drop_useless.columns if col not in ['SalePrice','Id']]
model.fit(pd.get_dummies(df_train_drop_useless[every_column_except_y]),df_train_drop_useless['SalePrice'])

OrderedDict(sorted(model.booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

#### 4.1.3 Feature extraction
* cut off by PCA (principal component analysis)

### 4.2 Understand relationship between features and house price
#### 4.2.1 Take a quick look for big picture

In [ ]:
# scatterplot
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars','GarageArea','TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train_drop_useless[cols], height = 2.5)

#### 4.2.2 Understand relationship between some numerical features and house price

* Ground living Area vs Sale Price

In [ ]:
# show relationship between SalePrice and Living Area
plt.subplots(figsize=(10,10))
sns.scatterplot(x=df_train_drop_useless['GrLivArea'],y=df_train_drop_useless['SalePrice'])

In [ ]:
## drop outliers of data which sale price > 13.5 and living area > 4000
drop_index = df_train_drop_useless[(df_train_drop_useless['SalePrice'] > 13.5) | (df_train_drop_useless['GrLivArea'] > 8.3)].index
df_train_drop_outliers = df_train_drop_useless.drop(drop_index)
print(df_train_drop_outliers.shape)

In [ ]:
## show relationship between living area and Sale price after droping 
plt.subplots(figsize=(10,10))
sns.scatterplot(x=df_train_drop_outliers['GrLivArea'],y=df_train_drop_outliers['SalePrice'],data=df_train)

* Area features VS Sale Price

In [ ]:
y_col_vals = 'SalePrice'
area_features = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                 'MasVnrArea', 'GarageArea', 'LotArea',
                 'WoodDeckSF', 'OpenPorchSF', 'BsmtFinSF1']
                # 'ScreenPorch'
x_col_vals = area_features

In [ ]:
nr_rows=3
nr_cols=3

fig = tools.make_subplots(rows=nr_rows, cols=nr_cols, print_grid=False,
                          subplot_titles=area_features )
                                                                
for row in range(1,nr_rows+1):
    for col in range(1,nr_cols+1): 
        
        i = (row-1) * nr_cols + col-1
     
        trace = go.Scatter(x = df_train_drop_useless[x_col_vals[i]], 
                           y = df_train_drop_useless[y_col_vals], 
                           name=x_col_vals[i], 
                           mode="markers", 
                           opacity=0.8)

        fig.append_trace(trace, row, col,)
 
                                                                                                  
fig['layout'].update(height=700, width=900, showlegend=False,
                     title='SalePrice vs. Area features')
iplot(fig)                                                

It shows that all of the Area features are positive related to sale price. In the Pearson correlation coefficient, it shows that Total Basement SF having very strong positive relation with first floor SF and we also obvered that Total Basement SF having strong positive relation with second floor SF. So I think it should have new feature sum all of area features.

In [ ]:
df_train_drop_outliers['allarea'] = (df_train_drop_outliers['TotalBsmtSF'] + df_train_drop_outliers['1stFlrSF'] + df_train_drop_outliers['2ndFlrSF'] + df_train_drop_outliers['MasVnrArea']
                                    + df_train_drop_outliers['GarageArea']  + df_train_drop_outliers['WoodDeckSF'] + df_train_drop_outliers['OpenPorchSF']
                                    + df_train_drop_outliers['BsmtFinSF1'].astype(int))

df_test_drop['allarea'] = (df_test_drop['TotalBsmtSF'] + df_test_drop['1stFlrSF'] + df_test_drop['2ndFlrSF'] + df_test_drop['MasVnrArea']+ df_test_drop['GarageArea']
                        + df_test_drop['WoodDeckSF'] + df_test_drop['OpenPorchSF']+ df_test_drop['BsmtFinSF1'].astype(int))

In [ ]:
print(df_train_drop_outliers['allarea'].corr(df_train_drop_outliers['SalePrice']))

* Groud Living Area VS Total rooms above grade

In [ ]:
trace = go.Scatter(x = df_train_drop_useless['GrLivArea'],
                  y = df_train_drop_useless['TotRmsAbvGrd'],
                  mode = 'markers',
                  opacity = 0.6)
data = [trace]
layout = go.Layout(
    title = 'Ground Living Area vs Total Rooms Grade',
    xaxis = dict(
        title = 'Ground Living Area',
        ticklen = 5,
        ),
    yaxis = dict(
        title = 'Total Rooms Grade',
        ticklen = 5,
        ),
)
fig = go.Figure(data=data,layout=layout)
iplot(fig)


#### 4.2.3 Understand relationship between some categorical features and house price

* Total Rooms Grade vs Ground Living Area

In [ ]:
trace = []
for name,group in df_train_drop_useless[['GrLivArea','TotRmsAbvGrd']].groupby('TotRmsAbvGrd'):
    trace.append(go.Box(y = group['GrLivArea'].values, 
                        boxpoints = 'all',
                        name = name))
layout = go.Layout(
    title = 'Total Rooms Grade vs Ground Living Area',
    xaxis = dict(
        title = 'Total Rooms Above Grade',
        ticklen = 5),
    yaxis = dict(
        title = 'Ground Living Area',
        ticklen = 5))
fig = go.Figure(data=trace,layout=layout)
iplot(fig)

* Garage Car vs Garage Area

In [ ]:
trace = []
for name,group in df_train_drop_useless[['GarageCars','GarageArea']].groupby('GarageCars'):
    trace.append(go.Box(y = group['GarageArea'], name = name))
layout = go.Layout(
                title = 'GarageCars vs GarageArea',
                xaxis = dict(
                    title = 'Garage Cars',
                    ticklen = 5),
                yaxis = dict(
                    title = 'Garage Area',
                    ticklen = 5))
fig =go.Figure(data=trace,layout=layout)
iplot(fig)

* Over all quality VS Sale Price

In [ ]:
## show relationship between all quality and Sale price
f,ax = plt.subplots(figsize=(10,10))
sns.boxplot(x=df_train_drop_useless['OverallQual'],y=df_train_drop_useless['SalePrice'],data=df_train)

* Year Built VS House Price

In [ ]:
## show relationship between year built of house and sale price
f,ax = plt.subplots(figsize=(20,10))
sns.boxplot(x=df_train_drop_useless['YearBuilt'],y=df_train_drop_useless['SalePrice'])
plt.xticks(rotation='90')

In [ ]:
## show relationship between year built of house and sale price group by overall quality
